# Train Bart-L with SFT
Steps to run:
* Add the following to secret keys: Github PAT, HF Token (write access)


In [1]:
import os
import sys
from google.colab import userdata

# ADD A PAT https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')  # put the token in colab secret keys
os.environ['REPOSITORY'] =  "github.com/jth500/maet-pln.git"
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
!git clone -b refactor https://${{GITHUB_TOKEN}}@${{REPOSITORY}}
!mv maet-pln maet_pln # rename to remove the invalid dash
sys.path.append("maet_pln/src")

Cloning into 'maet-pln'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 547 (delta 67), reused 79 (delta 41), pack-reused 416
Receiving objects: 100% (547/547), 624.41 KiB | 1.94 MiB/s, done.
Resolving deltas: 100% (340/340), done.


In [2]:
!pip install -q pyarrow==12.0.0
# !pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q transformers
!pip install -q tdqm torch
!pip install -q -U bitsandbytes
!pip install -q -U evaluate
# !pip install -q evaluate rouge_score
!pip install python-dotenv
!pip install peft
!pip install cohere
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 32.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

In [3]:
import random
import torch
from datasets import (load_dataset,
                      Dataset)
from tqdm import tqdm
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          Trainer,
                          GenerationConfig,
                          pipeline)
import evaluate


from datasets import get_dataset_split_names
import json
import pandas as pd

In [4]:
from data_handler.data_handler import BARTDatasetHandler

In [8]:
DATASET_NAME = "EdinburghNLP/xsum"
dh = BARTDatasetHandler(DATASET_NAME, tokenizer=None, data_size=100, push_to_hub=True, save_locally=False, rlaif=False)
data = dh.process_data()
tokenizer = dh.tokenizer

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1191 > 1024). Running this sequence through the model will result in indexing errors


Filter:   0%|          | 0/90 [00:00<?, ? examples/s]

ERROR:data_handler.data_handler:Couldn't push the data to HF. Is HF_UN  set in .env?
Traceback (most recent call last):
  File "/content/maet_pln/src/data_handler/data_handler.py", line 178, in save_dataset
    d.push_to_hub(f"{os.environ['HF_UN']}/{self.ID}_{k}")
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'HF_UN'


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

ERROR:data_handler.data_handler:Couldn't push the data to HF. Is HF_UN  set in .env?
Traceback (most recent call last):
  File "/content/maet_pln/src/data_handler/data_handler.py", line 178, in save_dataset
    d.push_to_hub(f"{os.environ['HF_UN']}/{self.ID}_{k}")
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'HF_UN'


In [10]:
sft_train_data, val_data = data
print([len(dataset) for dataset in data])

[80, 9]


In [12]:
### SFT
from sft import SFT
from model_builder import BARTModelBuilder
import torch
torch.device("cuda")



device(type='cuda')

In [15]:
# SET UP BASE MODEL
save_dir = "sft-bart-xsum-0504"
base_model_id = "facebook/bart-large"
model_builder = BARTModelBuilder(base_model_id, tokenizer=tokenizer)
base_model = model_builder.base_model
stf_trainer = SFT(
    save_dir=save_dir,
    tokenizer=tokenizer,
    base_model=model_builder.base_model,
    train_dataset=sft_train_data
)

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [16]:
torch.cuda.empty_cache()
stf_trainer.train_model()
stf_trainer.push_model_to_hub()
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1834: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case jth500/sft-bart-xsum-0504).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
32,2.992400
64,1.613600
96,1.326900
128,1.078100
160,0.827200


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

# Inference

In [17]:
from  model_builder import BARTModelBuilder
from transformers import pipeline

In [18]:
summarizer = pipeline('summarization', model="jth500/" + save_dir, tokenizer=tokenizer)

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

In [26]:
# TEMPLATE = (
#     """<s>You are an expert in text summarization. You are given the full text."""
#     """Your job is to summarise the text as concisely and accurately as possible in a single sentence.\n\n"""
#     """### TEXT:\n{input}\n\n"""
#     """### SUMMARY:</s>"""
# )
# prompts = [TEMPLATE.format(input=i) for i in val_data['input']]
prompts = list(val_data['input'])
prompts[1]

'<s>You are an expert in text summarization. You are given the full text.Your job is to summarise the text as concisely and accurately as possible in a single sentence.\n\n### TEXT:\nBig oil producing nations meet this weekend in Qatar to discuss plans for a freeze in production levels.\nBut there is scepticism over whether such a deal would make much difference to the current oversupply of oil in the market.\nThe Dow Jones Industrial Average fell 28.97 points to 17,897.46.\nThe S&P 500 was down 2.05 points to 2,080.73. The technology focused Nasdaq Composite slipped 7.67 points to 4,938.22.\nStocks in the energy sectors fell as another US bank - Citigroup - reported it was setting cash aside to cover losses on loans to the energy industry.\nShares in banking giant Citigroup closed 0.13% down after trading mostly higher all day. It reported a 27% fall in quarterly profit, but that was not as bad as analysts had been expecting.\nShares of Marathon Petroleum were down 3.8%, Exxon Mobile 

In [33]:
responses = []
summaries = []
indices = []

for i, p in enumerate(tqdm(prompts)):
  try:
    r = summarizer(p, max_length=10000, truncation=True)
    responses.append(r[0]['summary_text'])
    summaries.append(val_data[i]['output'])
    indices.append(i)
  except:
    pass

100%|██████████| 9/9 [08:09<00:00, 54.39s/it]


In [34]:
df = pd.DataFrame({'summary_pred': responses, 'summary_actual': summaries, 'index': indices,
                   'input': val_data[indices]['input']})
df.head()

,summary_pred,summary_actual,index,input
0,A former Royal Navy officer has told an inques...,<s>A boyfriend of a young soldier who died at ...,0,<s>You are an expert in text summarization. Yo...
1,The US stock market has closed lower for the s...,<s>(Close):Wall Street markets fell on Friday ...,1,<s>You are an expert in text summarization. Yo...
2,Bayern Munich midfielder Marco Reus has been l...,<s>Borussia Dortmund's Marco Reus has been omi...,2,<s>You are an expert in text summarization. Yo...
3,Monaco head coach Leonardo Jardim has announce...,<s>Monaco boss Leonardo Jardim has been reward...,3,<s>You are an expert in text summarization. Yo...
4,Wales produced a strong performance to overcom...,<s>Wales held on for a nervy win over Argentin...,4,<s>You are an expert in text summarization. Yo...
